# Week 7 Exercises

This week we'll be exploring computer vision & neural networks by training a convolutional neural network (CNN) on ADDI (Automated computer-based Diagnosis system for Dermascopic Images) dataset. You are able to run this notebook on GPU to speed up training and predictions, since we are building a larger model than before (in the toolbar, click Edit->Notebook settings->Hardware accelerator). Run the below cell to get started (may take a few minutes to download data).

In [0]:
import os, scipy.ndimage
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import requests
import zipfile
import io

# Download and extract data.
r = requests.get("http://web.stanford.edu/class/cs21si/resources/unit4_resources.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

base_path = 'unit4_resources/addi/'
train_path = base_path + 'train'
val_path = base_path + 'val'
test_path = base_path + 'test'

## Exercise 1: Visualize the Data

Check out some of the images from the dataset by running the code below! Feel free to change the indices ("*idx*") to look through other images.

In [0]:
def show_example(label, idx=0):
    img_name = os.listdir('%s/%s' % (train_path, label))[idx]
    path = '%s/%s/%s' % (train_path, label, img_name)
    img = plt.imread(path)
    print('Showing %s image with index %d' % (label, idx))
    plt.figure()
    plt.imshow(img)
    plt.show()

show_example('normal', idx=0), show_example('abnormal', idx=0)

## Exercise 2: Train a Simple CNN

Now, we'll create a simple CNN model with just convolutional and fully-connected layers on the task. Much of the code is heavily influenced by [the Keras CNN tutorial](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), so definitely check that out!

### Load the data
Below, we have code that loads the data as an ImageDataGenerator, which will make it convenient for the Keras framework to deal with. Notice that the train generator has more input parameters--these are called data augmentations and allow us to generalize better when we have little training data. Read more about data augmentation [here](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced).

Run the code below to load the data.

In [0]:
def load_data(batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_path, 
                                                        target_size=(150, 150),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

    val_generator = test_datagen.flow_from_directory(val_path,
                                                            target_size=(150, 150),
                                                            batch_size=batch_size,
                                                            class_mode='binary')
    
    test_generator = test_datagen.flow_from_directory(test_path,
                                                      target_size=(150, 150),
                                                      batch_size=batch_size,
                                                      class_mode='binary')
    
    return train_generator, val_generator, test_generator

train_generator, val_generator, test_generator = load_data(16)

### Create your Model

Write the code for your simple CNN below. Check out the [Keras documentation on CNNs](https://keras.io/layers/convolutional/) for reference!

In [0]:
def get_simple_cnn():
    ### YOUR CODE HERE ###
    model = Sequential()

    
    ### END CODE ###
    
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

simple_model = get_simple_cnn()
simple_model.summary()

### Train the Model!

Run the code below to train your simple model on the training data, evaluating on the validation data at every epoch. You should get around 60% on the validation set with this model.

In [0]:
def train_model(model, train_generator, validation_generator, epochs=5):
    model.fit_generator(train_generator,
                        steps_per_epoch=len(train_generator),
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=len(validation_generator))
    
train_model(simple_model, train_generator, val_generator, epochs=10)

## Exercise 3: Train an Advanced CNN

Now, we'll create a more advanced CNN model by adding max-pooling and spatial batch-normalization.

### Create your Model

Architect your advanced model below. Again, be sure to reference the [Keras CNN docs](https://keras.io/layers/convolutional/), as well as the [Keras Normalization docs](https://keras.io/layers/normalization/) (for the Spatial Batch Norm layer).

In [0]:
def get_advanced_cnn():
    ### YOUR CODE HERE ###
    model = Sequential()
    


    
    ### END CODE ###
    
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

advanced_model = get_advanced_cnn()
advanced_model.summary()

### Train the Model!

Run the code below to train your advanced model on the training data, evaluating on the validation data at every epoch. You should be able to get around 70% accuracy on validation with this model.

In [0]:
train_model(advanced_model, train_generator, val_generator, epochs=10)

### Evalutate the Model

Evaluate your model on the test set below! The first number is the loss, and the second is the accuracy. You should be able to get around 70% accuracy on the test set with your advanced model, but this will vary significantly due to randomness!

In [0]:
advanced_model.evaluate_generator(test_generator, len(test_generator))

Great work! For homework, we'll see that our model is susceptible to adversarial attacks, images that look the same to humans but are perturbed in a very particular way in order to fool our model.